In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In questa lezione, vedremo come usare Keras e le due API che offre per creare dei modelli per il deep learning.

## Lettura dei dati

Al solito, come primo step, carichiamo in memoria i dati che vogliamo analizzare. In questo caso, useremo uno dei dataset forniti con Keras ([qui](https://keras.io/api/datasets/) un elenco completo), ovvero MNIST, estremamente utilizzato come semplice dataset per l'image recognition.

Una volta caricato il dataset, usando la funzione `load_data` del package `keras.dataset.mnist`, andremo ad effettuare alcune semplici operazioni di preprocessing; in particolare:

* normalizzeremo i valori assunti dai pixel delle singole immagini in modo che ricadano tra 0 ed 1;
* useremo la funzione [`expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html) di NumPy per fare in modo che sia aggiunta una nuova dimensione ai dati, in modo che rispettino il parametro `input_shape`, rappresentativo della dimensione dell'array attesa in ingresso dalla rete;
* infine, convertiremo le label in dati *categorical*, mediante la funzione `to_categorical` contenuta nel package `keras.utils`. In particolare, questa funzione effettua il *one-hot encoding* del vettore passato in ingresso.

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
print('Dimensioni di X prima dell\'uso di expand_dims: {}'.format(X_train.shape))
X_train = np.expand_dims(X_train, -1)
print('Dimensioni di X dopo l\'uso di expand_dims: {}'.format(X_train.shape))
X_test = np.expand_dims(X_test, -1)
print('\nForma dei primi cinque elementi di y_train prima di to_categorical: \n{}'.format(y_train[:5]))
y_train = keras.utils.to_categorical(y_train, num_classes)
print('Forma del primo elemento di y_train dopo to_categorical: \n{}'.format(y_train[:5,:]))
y_test = keras.utils.to_categorical(y_test, num_classes)

Dimensioni di X prima dell'uso di expand_dims: (60000, 28, 28)
Dimensioni di X dopo l'uso di expand_dims: (60000, 28, 28, 1)

Forma dei primi cinque elementi di y_train prima di to_categorical: 
[5 0 4 1 9]
Forma del primo elemento di y_train dopo to_categorical: 
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## La Sequential API

Abbiamo già accennato al fatto che Keras offra due diverse possibilità per la creazione di un modello di rete neurale.

La prima che tratteremo è la **Sequential API**, che modella la rete come una sequenza (lineare) di layer, ognuno dei quali ha esattamente un tensore di ingresso ed un tensore di uscita.

Ad esempio, possiamo creare una rete (di dimensioni volutamente ridotte) concatenando, ad un layer di `Input`, due sequenze di convoluzione e max pooling, un layer `Flatten` ed il classico layer di classificazione (`Dense` con `activation='softmax'`).

In [3]:
smodel = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(num_classes, activation='softmax')
])

Possiamo rivedere la struttura di questo modello mediante la funzione `summary()`.

In [4]:
smodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
____________________________________________________

Possiamo adesso usare la funzione `compile` per finalizzare il modello, specificando le funzioni di costo e l'ottimizzatore da usare, e poi addestrarlo sui dati a nostra disposizione usando il metodo `fit`.

In [5]:
smodel.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

smodel.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
938/938 [==============================] - 13s 13ms/step - loss: 0.5459 - accuracy: 0.8408
Epoch 2/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0769 - accuracy: 0.9762
Epoch 3/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0564 - accuracy: 0.9829
Epoch 4/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0432 - accuracy: 0.9871
Epoch 5/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0389 - accuracy: 0.9883
Epoch 6/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0316 - accuracy: 0.9906
Epoch 7/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0262 - accuracy: 0.9924
Epoch 8/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0235 - accuracy: 0.9930
Epoch 9/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0204 - accuracy: 0.9935
Epoch 10/10
938/938 [==============================] - 13s 14ms/step - loss: 0.017

## La Functional API

La seconda possibilità offertaci da Keras è quella di usare la **Functional API**, che ci offre maggior controllo sul modello da creare, al prezzo di una sintassi leggermente più complessa.

Infatti, la Functional API si presta ad un uso decisamente più avanzato rispetto alla Sequential API: laddove quest'ultima si limita ad "impilare" i layer uno dietro l'altro, la Functional API basa il suo funzionamento sul concetto di *grafo aciclico diretto*, permettendo quindi topologie più complesse con layer condivisi, input ed output multipli, e via discorrendo.

In questo corso, ci focalizzeremo sulla Sequential API; tuttavia, a scopo illustrativo, vediamo come creare un modello analogo al precedente mediante la Functional API.

Per prima cosa, dobbiamo definire un input, usando al solito l'apposito layer.

In [6]:
inputs = layers.Input(shape=input_shape)

Ora però possiamo apprezzare la differenza tra la Sequential API e la Functional API. Infatti, quest'ultima ci permette di aggiungere un nodo al grafo dei layer che compongono la rete neurale *chiamando un layer sull'oggetto `inputs`*, il che equivale a "creare un collegamento" tra `inputs` ed il nuovo layer.

In [7]:
x = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)

Questa operazione va ripetuta fino a raggiungere il layer `outputs`, che sarà quello della classificazione mediante `softmax`.

In [8]:
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

Usiamo ora il metodo `Model` per definire input ed output del modello, ed assegnamoli un nome. Verifichiamo inoltre con `summary` che la struttura sia coerente con quella vista in precedenza.

In [9]:
fmodel = keras.Model(inputs=inputs, outputs=outputs, name='functional_model')
fmodel.summary()

Model: "functional_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)           

Una nota: vediamo che, a differenza del `summary` del modello ottenuto mediante Sequential API, qui siamo in grado di vedere la forma del layer di input. Questo effetto è *voluto*.

In ultimo, compiliamo il modello ed addestriamolo alla stessa maniera del precedente.

In [10]:
fmodel.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

fmodel.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
938/938 [==============================] - 13s 14ms/step - loss: 0.5355 - accuracy: 0.8474
Epoch 2/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0732 - accuracy: 0.9784
Epoch 3/10
938/938 [==============================] - 14s 14ms/step - loss: 0.0513 - accuracy: 0.9845
Epoch 4/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0441 - accuracy: 0.9867
Epoch 5/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0357 - accuracy: 0.9887
Epoch 6/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0313 - accuracy: 0.9905
Epoch 7/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0238 - accuracy: 0.99250s - loss: 0.0237 - ac
Epoch 8/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0226 - accuracy: 0.9925
Epoch 9/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0197 - accuracy: 0.9937
Epoch 10/10
938/938 [==============================] - 13s 1

## Conclusioni

In questa lezione, abbiamo visto come usare le Functional e le Sequential API di Keras per ottenere un modello di rete da addestrare. Nelle prossime lezioni, vedremo meglio alcuni tipi di layer che è possibile usare, ed alcune tecniche che possiamo sfruttare per migliorare le performance della rete neurale.